In [18]:
from IPython.lib import kernel

kernel.get_connection_file()

'/run/user/1000/jupyter/kernel-baa51188-70a0-43db-bb17-c39b6c82458e.json'

In [ ]:
from gensim.corpora import Dictionary
from utilities.LabeledDocuments import LabeledDocument
from gensim import corpora
from os.path import isfile

# inspired by http://radimrehurek.com/gensim/tut1.html#corpus-formats

class MyCorpus():
    def __init__(self, labeledDocs='samples', \
                 dictFile='data/MMCorpus/econstor.dict', \
                 corpusFile='data/MMCorpus/corpus.mm'):
        
        if isfile(dictFile):
            self.dictionary = Dictionary.load(dictFile)
        else:
            labeledDocs = LabeledDocument(wd=labeledDocs)
            #because LabeledDocument yields gensim.models.doc2vec.LabeledSentence
            # objects and we are only interested in the words property
            docGen = (d.words for d in labeledDocs)
            dictionary = Dictionary(iter(docGen))
            dictionary.save(dictFile)
            self.dictionary = dictionary
            
        if isfile(corpusFile):
            self.corpus = corpora.MmCorpus(corpusFile)
        else:
            corpora.MmCorpus.serialize(corpusFile, self)
            self.corpus = corpora.MmCorpus(corpusFile)
            
    
    def __getitem__(self, name):
        if name == 'dictionary':
            return self.dictionary
        elif name == 'corpus':
            return self.corpus
        else:
            raise AttributeError('This object has no attribute {}.'.format(name))
    
    def __len__(self):
        return len(self.corpus)
    
    def __iter__(self):
        labeledDocs = LabeledDocument(wd='samples/')
        for d in labeledDocs:
            yield self.dictionary.doc2bow(d.words)

In [ ]:
%%timeit
from gensim.models.ldamulticore import LdaMulticore 

LDAFile = 'data/LDA/test.lda'

if isfile(LDAFile):
    lda = LdaMulticore.load(LDAFile)
else:
    myCorpus = MyCorpus()
    lda = LdaMulticore(myCorpus, id2word=myCorpus.dictionary,\
                       num_topics=25, workers=4, passes=3, \
                       batch=True)
    lda.save(LDAFile)

INFO:gensim.utils:loading Dictionary object from data/MMCorpus/econstor.dict
INFO:gensim.corpora.indexedcorpus:loaded corpus index from data/MMCorpus/corpus.mm.index
INFO:gensim.matutils:initializing corpus reader from data/MMCorpus/corpus.mm
INFO:gensim.matutils:accepted corpus with 74150 documents, 2259478 features, 104142524 non-zero entries
INFO:gensim.models.ldamodel:using symmetric alpha at 0.04
INFO:gensim.models.ldamodel:using symmetric eta at 0.04
INFO:gensim.models.ldamodel:using serial LDA version on this node
INFO:gensim.models.ldamulticore:running batch LDA training, 25 topics, 3 passes over the supplied corpus of 74150 documents, updating every 74150 documents, evaluating every ~74150 documents, iterating 50x with a convergence threshold of 0.001000
INFO:gensim.models.ldamulticore:training LDA model using 4 processes
INFO:gensim.models.ldamulticore:PROGRESS: pass 0, dispatched chunk #0 = documents up to #2000/74150, outstanding queue size 1


In [ ]:
lda.show_topics()